In [3]:
import sqlite3

def get_table_names(file_path):
    
    # Create a SQL connection to our SQLite database
    con = sqlite3.connect(file_path)

    # creating cursor
    cur = con.cursor()

    # reading all table names
    table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table' OR type = 'view'")]

    # Be sure to close the connection
    con.close()

    table_names = [name[0] for name in table_list]

    return table_names


def print_data(file_path, table_name = None):
    
    if table_name is None:
        table_names = get_table_names(file_path)

        # Create a SQL connection to our SQLite database
        con = sqlite3.connect(file_path)

        cur = con.cursor()

        # The result of a "cursor.execute" can be iterated over by row
        for name in table_names:
            if name == 'ElementaryGeometries' or name == 'SpatialIndex':
                continue
            sql_command = "SELECT * FROM " + name
            print(name)
            for row in cur.execute(sql_command):
                print(row)
            print('\n\n')

        # Be sure to close the connection
        con.close()
    
    else:
        # Create a SQL connection to our SQLite database
        con = sqlite3.connect(file_path)

        cur = con.cursor()

        # The result of a "cursor.execute" can be iterated over by row
        sql_command = "SELECT * FROM " + table_name
        print(table_name)
        for row in cur.execute(sql_command):
            print(row)

        # Be sure to close the connection
        con.close()

In [23]:
a = get_table_names(r"\Users\Administrator\Downloads\DP08_XX_K01_C07_F2_N001.db")
len(a)
print(a)

['__EFMigrationsHistory', 'spatial_ref_sys', 'spatialite_history', 'sqlite_sequence', 'geometry_columns', 'geom_cols_ref_sys', 'spatial_ref_sys_aux', 'spatial_ref_sys_all', 'views_geometry_columns', 'virts_geometry_columns', 'geometry_columns_statistics', 'views_geometry_columns_statistics', 'virts_geometry_columns_statistics', 'geometry_columns_field_infos', 'views_geometry_columns_field_infos', 'virts_geometry_columns_field_infos', 'geometry_columns_time', 'geometry_columns_auth', 'views_geometry_columns_auth', 'virts_geometry_columns_auth', 'vector_layers', 'vector_layers_auth', 'vector_layers_statistics', 'vector_layers_field_infos', 'sql_statements_log', 'SpatialIndex', 'ElementaryGeometries', 'Project', 'Document', 'DocumentParameter', 'Element', 'Workset', 'ElementParameterInteger', 'ElementParameterNumber', 'ElementParameterRevitId', 'ElementParameterText', 'LevelTypeElements', 'LevelElements', 'ModelLineElements', 'RoomElements']


In [1]:
import sqlite3

def initialize(file_path):
    
    # Create a SQL connection to our SQLite database
    # returns connection and cursor

    con = sqlite3.connect(file_path)
    cur = con.cursor()
    print('Opening connection')

    return cur, con


def finilize(con):

    # Closes SQL connection

    con.close()
    print('Closing connection')

In [6]:
def work(file_path, sql):
    
    cur, con = initialize(file_path)
    out = cur.execute(sql)
    print('SQL done')
    
    table_list = [a[0] for a in out] #why do I get ordered output sometimes???
    print('Work completed')

    finilize(con)

    return table_list

In [10]:
sql = "SELECT DISTINCT Area FROM RoomElements LIMIT 10"
table_list = work(r"\Users\Administrator\Downloads\DP08_XX_K01_C07_F2_N001.db", sql)

print(table_list)

Opening connection
SQL done
Work completed
Closing connection
[411.04770086900044, 1478.7482757795442, 1071.8324017396244, 3686.413271557628, 222.6724220989704, 575.1817269431986, 355.6055593861197, 554.7101625277137, 527.8862984124022, 420.27662310201276]


In [37]:
import sqlite3

def generate_docs(db, docs, title):
    # open doc file and define table template
    f = open(docs, "w+")
    f.write(title + "\n")
    table_template = "|{}|{}|\n| :-: | :-:|\n".format("COLUMN", "DESCRIPTION")
    # open database
    con = sqlite3.connect(db)
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='view';")
    for table in cur.fetchall():
        # print table name
        table_name = table[0].upper()
        f.write("### {}\n".format(table_name))
        f.write(table_template)
        if table_name == 'ELEMENTARYGEOMETRIES' or table_name == 'SPATIALINDEX':
            continue
        # print table columns
        cur.execute("SELECT * FROM {};".format(table_name))
        cols = [description[0] for description in cur.description]
        for col in cols:
            f.write("| {} | |\n".format(col))
    f.close()


generate_docs(r"\Users\Administrator\Downloads\DP08_XX_K01_C07_F2_N001.db", "db_template_views.md",
              "Room schema")

In [16]:
# issue with SpatialIndex and ElementaryGeometries
print_data(r"\Users\Administrator\Downloads\DP08_XX_K01_C07_F2_N001.db", 'SpatialIndex')

SpatialIndex


OperationalError: no such module: VirtualSpatialIndex

In [8]:
import pymysql
  
def mysqlconnect():
    # To connect MySQL database
    conn = pymysql.connect(
        host='192.168.2.140',
        user='admin', 
        password = "admin",
        db='db',
        port=6606
        )
      
    cur = conn.cursor()
    cur.execute("select @@version")
    output = cur.fetchall()
    print(output)
      
    # To close the connection
    conn.close()

In [9]:
mysqlconnect()

(('5.7.37',),)


In [20]:
#import os
import pymysql
import pandas as pd

""" host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')
 """

conn = pymysql.connect(
    host='192.168.2.140',
    user='admin', 
    password = "admin",
    db='db',
    port=6606
    )

#df = pd.read_sql_query("SELECT * FROM folders", conn)

In [18]:
pd.read_sql_query("SELECT table_name FROM information_schema.tables", conn)

,table_name
0,CHARACTER_SETS
1,COLLATIONS
2,COLLATION_CHARACTER_SET_APPLICABILITY
3,COLUMNS
4,COLUMN_PRIVILEGES
...,...
64,items
65,objects
66,projects
67,properties
